# Fine-Tune Phi-2 on Amazon SageMaker

This notebook demonstrates how to fine-tune the Microsoft Phi-2 model from Hugging Face using Amazon SageMaker.

## Model Description

Phi-2 is a Transformer model with 2.7 billion parameters. It was trained using a combination of the data sources used for Phi-1.5, along with additional NLP synthetic texts and curated websites. Evaluations against benchmarks for common sense, language understanding, and logical reasoning indicate that Phi-2 achieves nearly state-of-the-art performance among models with fewer than 13 billion parameters.

For those seeking a large language model that offers a balance of lightweight design and broad capability, Phi-2 could be an attractive option.

## Fine-tuning task
Here we are fine-tuning the Phi-2 model using summarization samples from the Dolly dataset from Huggingface Hub. The goal is to improve the model's overall summarization capability.

## 1. Setup Development Environment

Our first step is to install Hugging Face Libraries we need on the client to correctly prepare our dataset and start our training/evaluations jobs. 

In [ ]:
!pip install transformers "datasets[s3]==2.18.0" "sagemaker>=2.190.0" --upgrade --quiet

## 2. Import and prepare the dataset

In [ ]:
import boto3
import sagemaker

sagemaker_session_bucket = None
sess = sagemaker.Session()

if sagemaker_session_bucket is None and sess is not None:
    sagemaker_session_bucket = sess.default_bucket()

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")



We are going to use [trl](https://huggingface.co/docs/trl/en/index) for fine-tuning, which supports popular instruction and conversation dataset formats. This means we only need to convert our dataset to one of the supported formats and `trl` will take care of the rest. Those formats include:

- conversational format

```json
{"messages": [{"role": "system", "content": "You are..."}, {"role": "user", "content": "..."}, {"role": "assistant", "content": "..."}]}
{"messages": [{"role": "system", "content": "You are..."}, {"role": "user", "content": "..."}, {"role": "assistant", "content": "..."}]}
{"messages": [{"role": "system", "content": "You are..."}, {"role": "user", "content": "..."}, {"role": "assistant", "content": "..."}]}
```

- instruction format

```json
{"prompt": "<prompt text>", "completion": "<ideal generated text>"}
{"prompt": "<prompt text>", "completion": "<ideal generated text>"}
{"prompt": "<prompt text>", "completion": "<ideal generated text>"}
```

In our example we are going to load open-source `dolly` dataset using the 🤗 Datasets library and then convert it into the the conversational format, where we include the schema definition in the system message for our assistant. We'll then save the dataset as jsonl file, which we can then use to fine-tune our model. 

In [ ]:
from datasets import load_dataset

# convert dataset to OAI messages
system_message = """You are a text summarizer. Users will provide you a text in English and you will generate a summary based on the provided SCHEMA.
SCHEMA:
{schema}"""

def create_conversation(sample):
  return {
    "messages": [
      {"role": "system", "content": system_message.format(schema=sample["instruction"])},
      {"role": "user", "content": sample["context"]},
      {"role": "assistant", "content": sample["response"]}
    ]
  }

# load dataset from the hub. Here we are using Dolly dataset from Databricks via Huggingface hub.
# databricks-dolly-15k is an open source dataset of instruction-following records.  
dolly_dataset = load_dataset("databricks/databricks-dolly-15k", split="train")

# filter the dataset to include only summarization examples
dataset = dolly_dataset.filter(lambda example: example["category"] == "summarization")
dataset = dataset.remove_columns("category")

# convert dataset to instruction format
dataset = dataset.map(create_conversation, batched=False)

print(dataset[0]["messages"])


After we processed the datasets we are going to use the [FileSystem integration](https://huggingface.co/docs/datasets/filesystems) to upload our dataset to S3. We are using the `sess.default_bucket()`, adjust this if you want to store the dataset in a different S3 bucket. We will use the S3 path later in our training script.

In [ ]:
import pandas as pd

# save training dataset to S3 using SageMaker session
s3_client = boto3.client('s3')

training_input_path = f's3://{sess.default_bucket()}/datasets/summarization'

df = pd.DataFrame(dataset)

# Convert DataFrame to JSON
json_data = df.to_json(orient='records')

# Upload JSON data to S3
key = 'datasets/summarization/train_dataset.json'
s3_client.put_object(Bucket=sess.default_bucket(), Key=key, Body=json_data)

print(f"Training data uploaded to:")
print(f"{training_input_path}/train_dataset.json")
print(f"https://s3.console.aws.amazon.com/s3/buckets/{sess.default_bucket()}/?region={sess.boto_region_name}&prefix={training_input_path.split('/', 3)[-1]}/")

## 3. Fine-Tune Phi-2 with QLoRA on Amazon SageMaker

We are now ready to fine-tune our model. We will use the [SFTTrainer](https://huggingface.co/docs/trl/sft_trainer) from `trl` to fine-tune our model. We will use the dataset formatting, packing and PEFT features in our example. 

As peft method we will use [QLoRA](https://arxiv.org/abs/2305.14314) a technique to reduce the memory footprint of large language models during finetuning, without sacrificing performance by using quantization. In Addition to QLoRA we will leverage the new [Flash Attention 2 integrationg with Transformers](https://huggingface.co/docs/transformers/perf_infer_gpu_one#flash-attention-2) to speed up the training. 

In [ ]:
# hyperparameters, which are passed into the training job
hyperparameters = {
  ### SCRIPT PARAMETERS ###
  'dataset_path': '/opt/ml/input/data/training/train_dataset.json', # path where sagemaker will save training dataset
  'model_id': "microsoft/phi-2",                     # Model ID from HuggingFace Hub
  'max_seq_len': 3072,                               # max sequence length for model and packing of the dataset
  'use_qlora': True,                                 # use QLoRA model
  ### TRAINING PARAMETERS ###
  'num_train_epochs': 3,                             # number of training epochs
  'per_device_train_batch_size': 1,                  # batch size per device during training
  'gradient_accumulation_steps': 4,                  # number of steps before performing a backward/update pass
  'gradient_checkpointing': True,                    # use gradient checkpointing to save memory
  'optim': "adamw_torch_fused",                      # use fused adamw optimizer
  'logging_steps': 10,                               # log every 10 steps
  'save_strategy': "epoch",                          # save checkpoint every epoch
  'learning_rate': 2e-4,                             # learning rate, based on QLoRA paper
  'bf16': True,                                      # use bfloat16 precision
  'tf32': True,                                      # use tf32 precision
  'max_grad_norm': 0.3,                              # max gradient norm based on QLoRA paper
  'warmup_ratio': 0.03,                              # warmup ratio based on QLoRA paper
  'lr_scheduler_type': "constant",                   # use constant learning rate scheduler
  'report_to': "tensorboard",                        # report metrics to tensorboard
  'output_dir': '/tmp/tun',                          # Temporary output directory for model checkpoints
  'merge_adapters': True,                            # merge LoRA adapters into model for easier deployment
}

In order to create a sagemaker training job we need an `HuggingFace` Estimator. The Estimator handles end-to-end Amazon SageMaker training and deployment tasks. The Estimator manages the infrastructure use. Amazon SagMaker takes care of starting and managing all the required ec2 instances for us, provides the correct huggingface container, uploads the provided scripts and downloads the data from our S3 bucket into the container at `/opt/ml/input/data`. Then, it starts the training job by running.

> Note: Make sure that you include the `requirements.txt` in the `source_dir` if you are using a custom training script. We recommend to just clone the whole repository.

In [ ]:
from sagemaker.huggingface import HuggingFace

# define Training Job Name 
job_name = 'phi2-7b-hf-text-to-sql-exp1'

# create the Estimator
huggingface_estimator = HuggingFace(
    entry_point          = 'run_sft.py',      # train script
    source_dir           = './trl',  # directory which includes all the files needed for training
    instance_type        = 'ml.g5.4xlarge',   # instances type used for the training job
    instance_count       = 1,                 # the number of instances used for training
    max_run              = 2*24*60*60,        # maximum runtime in seconds (days * hours * minutes * seconds)
    base_job_name        = job_name,          # the name of the training job
    role                 = role,              # Iam role used in training job to access AWS ressources, e.g. S3
    volume_size          = 300,               # the size of the EBS volume in GB
    transformers_version = '4.36',            # the transformers version used in the training job
    pytorch_version      = '2.1',             # the pytorch_version version used in the training job
    py_version           = 'py310',           # the python version used in the training job
    hyperparameters      =  hyperparameters,  # the hyperparameters passed to the training job
    disable_output_compression = True,        # not compress output to save training time and cost
)

> You can also use `g5.2xlarge` instead of the `g5.4xlarge` instance type, but then it is not possible to use `merge_weights` parameter, since to merge the LoRA weights into the model weights, the model needs to fit into memory. But you could save the adapter weights and merge them using [merge_adapter_weights.py](../scripts/merge_adapter_weights.py) after training.

We can now start our training job, with the `.fit()` method passing our S3 path to the training script.

In [ ]:
# define a data input dictonary with our uploaded s3 uris
data = {'training': training_input_path}

# starting the train job with our uploaded datasets as input
huggingface_estimator.fit(data, wait=True)

In our example for Phi-2, the SageMaker training job took `1497 seconds`, which is about `0.416 hours`. The ml.g5.4xlarge instance we used costs `$2.03 per hour` for on-demand usage in us-east-1 region. As a result, the total cost for training our Phi-2 model was only ~`$1`. 

## 4. Deploy & evaluate LLM on Amazon SageMaker and compare with the base model

In [ ]:
from sagemaker.huggingface import get_huggingface_llm_image_uri

# retrieve the llm image uri
llm_image = get_huggingface_llm_image_uri(
  "huggingface",
  version="1.4.0",
  session=sess,
)

# print ecr image uri
print(f"llm image uri: {llm_image}")

We can now create a `HuggingFaceModel` using the container uri and the S3 path to our model. We also need to set our TGI configuration including the number of GPUs, max input tokens. You can find a full list of configuration options [here](https://huggingface.co/docs/text-generation-inference/basic_tutorials/launcher).

In [ ]:
import json
from sagemaker.huggingface import HuggingFaceModel

# s3 path where the model will be uploaded
# if you try to deploy the model to a different time add the s3 path here
model_s3_path = huggingface_estimator.model_data["S3DataSource"]["S3Uri"]

# sagemaker config
instance_type = "ml.g5.2xlarge"
number_of_gpu = 1
health_check_timeout = 300

# define Model and Endpoint configuration parameter
config = {
  'HF_MODEL_ID': "/opt/ml/model", # path to where sagemaker stores the model
  'SM_NUM_GPUS': json.dumps(number_of_gpu), # Number of GPU used per replica
  'MAX_INPUT_LENGTH': json.dumps(1024), # Max length of input text
  'MAX_TOTAL_TOKENS': json.dumps(2048), # Max length of the generation (including input text)
}

# create HuggingFaceModel with the image uri
fine_tuned_model = HuggingFaceModel(
  role=role,
  image_uri=llm_image,
  model_data={'S3DataSource':{'S3Uri': model_s3_path,'S3DataType': 'S3Prefix','CompressionType': 'None'}},
  env=config
)

In [ ]:
# We will also deploy the base Phi-2 foundation model from Huggingface to compare the summarization performance

import json
import sagemaker
import boto3
from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri

try:
	role = sagemaker.get_execution_role()
except ValueError:
	iam = boto3.client('iam')
	role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

# Hub Model configuration. https://huggingface.co/models
hub = {
	'HF_MODEL_ID':'microsoft/phi-2',
	'SM_NUM_GPUS': json.dumps(1)
}

# create Hugging Face Model Class
huggingface_base_model = HuggingFaceModel(
	image_uri=get_huggingface_llm_image_uri("huggingface",version="1.4.2"),
	env=hub,
	role=role, 
) 


After we have created the HuggingFaceModel we can deploy it to Amazon SageMaker using the deploy method.

In [ ]:
# deploy fin-tuned model to an endpoint
tuned_llm = fine_tuned_model.deploy(
  initial_instance_count=1,
  instance_type=instance_type,
  container_startup_health_check_timeout=health_check_timeout, # 10 minutes to give SageMaker the time to download the model
)


# deploy base model to an endpoint
base_llm = huggingface_base_model.deploy(
  initial_instance_count=1,
  instance_type=instance_type,
  container_startup_health_check_timeout=health_check_timeout, # 10 minutes to give SageMaker the time to download the model
)

In [ ]:
inputs = "The Eglinton Crosstown LRT project, eagerly anticipated by many, faces ongoing delays as the head of the provincial transit agency Metrolinx highlights challenges with the software intended to control trains along the route. Metrolinx CEO Phil Verster recently addressed the project's status, acknowledging progress but emphasizing persistent issues with the software. Verster revealed that by June, the software will be undergoing its seventh iteration due to ongoing defects. Verster noted that while major construction is largely complete, there are still minor tasks remaining, such as addressing water leaks or fixing broken tiles. However, his primary concern lies with the software defects affecting the signalling and train control system, which are being addressed by contractors CTS and Alstom. Despite progress, Verster expressed that the pace of rectifying these defects is not as rapid as desired."

# send request
tuned_response = tuned_llm.predict({
	"inputs": inputs,
})

# send request
base_response = base_llm.predict({
	"inputs": inputs,
})
print (f"tuned_response: {tuned_response}")
print (f"base_response: {base_response}")


## Notice that the output from the tuned model (fine-tuned with summarization data) is more concise and is of better quality

## 5. Delete the endpoints and the dataset

In [ ]:
tuned_llm.delete_model()
tuned_llm.delete_endpoint()

base_llm.delete_model()
base_llm.delete_endpoint()

print(f"Deleted endpoint")

In [ ]:
import boto3

# Create an S3 client
s3_client = boto3.client('s3')

# Specify the bucket and file path
bucket_name = sess.default_bucket()
file_key = 'datasets/summarization/train_dataset.json'

# Delete the file
s3_client.delete_object(Bucket=bucket_name, Key=file_key)

print(f"Deleted training dataset file")